In [31]:
import os
import pandas as pd
os.chdir("/Users/anorawu/Documents/GitHub/CloudSeeding/data/炮台数据")

df = pd.read_csv('炮台数据_qwen.csv')


In [32]:
df.rename(columns={"微博内容": "weibo_text"},inplace=True)

In [33]:
os.chdir("/Users/anorawu/Documents/GitHub/CloudSeeding/data")
df_county = pd.read_stata('微博数据_with time and location.dta')

In [34]:
df_merge = pd.merge(df,df_county,how='inner',on='weibo_text')
df_merge = df_merge[['weibo_text','operation_year', 'operation_month', 'operation_day','prov', 'city', 'county','area']]

In [35]:
os.chdir("/Users/anorawu/Documents/GitHub/CloudSeeding/data/炮台数据")
df_merge.to_csv('炮台数据_merged_county.csv')

In [47]:
df_merge

,weibo_text,operation_year,operation_month,operation_day,prov,city,county,area
0,勐海县抓住有利天气条件 开展人工增雨作业有效缓解旱情记者黎智仙报道： 3月24日...,2020,3,20,云南省,西双版纳傣族自治州,勐海县,星火山、布朗山、勐混、曼稿保护区、勐邦水库
1,【甘肃民乐：人工增雨全县普降甘霖】6月以来，我县气温特高，降水特少，最长连续无降水日达19天...,2023,8,24,甘肃省,张掖市,民乐县,海潮坝、双树寺人影作业点
2,#桐梓天气# 【桐梓县气象台】2022年9月22日7时19分发布气象信息快报(单位:毫米):...,2022,9,21,贵州省,遵义市,桐梓县,官仓、九坝、花秋、天坪
3,#人工增雨#【青海雪下得那么大，原来与人工增雨有关】11月30日，我省各地大雪纷飞，你是否还...,2019,11,30,青海省,海南藏族自治州,共和县,共和、门源、乐都
4,#人工增雨#【青海雪下得那么大，原来与人工增雨有关】11月30日，我省各地大雪纷飞，你是否还...,2019,11,30,青海省,海东市,乐都区,共和、门源、乐都
...,...,...,...,...,...,...,...,...
12400,#内科大微分享# 【包头为什么要进行人工增雪？气象部门通过本报做出权威解释】2月13日夜间至...,2019,2,13,内蒙古自治区,包头市,白云鄂博矿区,固阳县、白云矿区至固阳县区域、阿塔山水库、达茂旗二愣滩、黄花滩水库
12401,#雪情通报# 1月18日20时-19日16时，在自然降雪和人工影响共同作用下，我市大部降中雪...,2020,1,18,陕西省,榆林市,,定边、靖边、榆阳、神木、府谷、吴堡
12402,#946快讯#【最大降雨量93.9毫米 今天起贵州天气“主打”雷阵雨系列】 在弱冷...,2020,3,2,贵州省,黔南布依族苗族自治州,都匀市,清镇、西秀、平坝、紫云、关岭、平塘、丹寨
12403,#946快讯#【最大降雨量93.9毫米 今天起贵州天气“主打”雷阵雨系列】 在弱冷...,2020,3,2,贵州省,黔东南苗族侗族自治州,凯里市,清镇、西秀、平坝、紫云、关岭、平塘、丹寨


In [57]:
import ollama
import pandas as pd
import csv
import logging
import sys
import json
import os

def get_qwen_response(city,area):
    # 构造prompt
    prompt = f"""目标：请帮助我根据“{city},{area}”返回所有具体位置信息，多条信息用逗号隔开。
    不要返回其他任何信息。
    例子1：city='上饶市',area='许村，中云等周边乡镇'，请返回：'上饶市许村，上饶市中云'。
    例子2：city='岳阳市‘,area=‘经开区康王，梅溪、君山柳林洲、南湖新区风雨山、临湘聂市镇、江南镇、长安街道’，请返回：
    '岳阳市经开区康王, 岳阳市梅溪，岳阳市君山柳林洲，岳阳市南湖新区风雨山，岳阳市临湘聂市镇，岳阳市江南镇，岳阳市长安街道'
    """

    response = ollama.chat(model="qwen2:7b", messages=[
        {
            "role": "user",
            "content": prompt,
        },
    ])
    result = response['message']['content']
    return result

df_merge_nodup = df_merge.drop_duplicates(subset=['weibo_text','city'])
df_merge_nodup['location'] = df_merge_nodup['area']


/var/folders/5h/rb902v091ys_0c5x0jt8sbjw0000gp/T/ipykernel_34660/3091379419.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merge_nodup['location'] = df_merge_nodup['area']


In [61]:
for index, row in df_merge_nodup.iterrows():
    if str(df_merge_nodup.loc[index,'area']).find("None") == -1:
        df_merge_nodup.loc[index,'location'] = get_qwen_response(df_merge_nodup.loc[index,'city'],df_merge_nodup.loc[index,'area'])
    else:
        df_merge_nodup.loc[index,'location'] = float('nan')
    print(df_merge_nodup.loc[index,'location'])
    

西双版纳傣族自治州星火山、西双版纳傣族自治州布朗山、西双版纳傣族自治州勐混、西双版纳傣族自治州曼稿保护区、西双版纳傣族自治州勐邦水库。
张掖市海潮坝、张掖市双树寺人影作业点
遵义市官仓, 遵义市九坝, 遵义市花秋, 遵义市天坪
海南藏族自治州,共和, 海南藏族自治州,门源, 海南藏族自治州,乐都
海东市共和，海东市门源，海东市乐都
海北藏族自治州, 海北藏族自治州共和, 海北藏族自治州门源, 海北藏族自治州乐都
阿坝藏族羌族自治州上杜柯作业点, 阿坝藏族羌族自治州茸木达作业点, 阿坝藏族羌族自治州南木达山顶作业点
张掖市山丹县人影办, 张掖市各作业点
根据目标描述，“滁州市,城郊作业点”这一信息中主要包含了城市名称“滁州市”和特定区域描述“城郊作业点”。由于具体的位置信息并未直接给出，我们可以理解为这个表述指向了该城市的某个具体的城郊地区作为作业点。

因此，按照例子中的输出格式要求，并基于给定的描述进行转换处理，“C滁州市,城郊作业点”可以转换为以下格式：

**C滁州市城郊作业点**

根据例子1和例子2的输出方式，我们可以将上述结果转换成：

**'C滁州市城郊作业点'**

请注意，由于实际的具体位置信息并未提供，上述输出已经是按照要求和逻辑做出的最佳解释。如果有更具体的位置列表或描述，请提供详细信息以便更精确地进行转换。
凉山彝族自治州,冕宁腊窝火场4个作业点，请返回：凉山彝族自治州冕宁腊窝火场1号作业点, 凉山彝族自治州冕宁腊窝火场2号作业点, 凉山彝族自治州冕宁腊窝火场3号作业点, 凉山彝族自治州冕宁腊窝火场4号作业点
上饶市许村, 上饶市中云
杭州市百丈、杭州市鸬鸟、杭州市径山、杭州市黄湖、杭州市良渚、杭州市塘栖、杭州市瓶窑、杭州市仁和以及老余杭。
聊城市,菜屯，贾寨，博平，冯屯(王老)，杜郎口，乐平(郝集)
亳州市双沟，亳州市芦庙，亳州市标里，亳州市涡南，亳州市王人，亳州市西潘楼，亳州市坛城，亳州市小涧
聊城市杜郎口，聊城市韩屯，聊城市菜屯，聊城市贾寨，聊城市博平，聊城市冯屯，聊城市乐平
昆明市车木河水库, 昆明市凤仪水库
凉山彝族自治州太和乡，西昌市，越西，雷波，冕宁，喜德
昆明市撒营盘宜岔, 昆明市十一电站, 昆明市安宅
昆明市云龙水库管理所, 昆明市安宅, 昆明市云龙水城河, 撒营盘宜岔, 双化水库, 十一电站
内江市普润作业点
昆明市大尖山

In [70]:
import re
import numpy as np

for index, row in df_merge_nodup.iterrows():
    l = re.split(r'[,.，、。：:]', str(df_merge_nodup.loc[index,'location']))
    l = [item for item in l if item]
    print(l)
    df_merge_nodup.at[index,'location'] = np.array(l)

df_merge_nodup = df_merge_nodup.explode('location')


['西双版纳傣族自治州星火山', '西双版纳傣族自治州布朗山', '西双版纳傣族自治州勐混', '西双版纳傣族自治州曼稿保护区', '西双版纳傣族自治州勐邦水库']
['张掖市海潮坝', '张掖市双树寺人影作业点']
['遵义市官仓', ' 遵义市九坝', ' 遵义市花秋', ' 遵义市天坪']
['海南藏族自治州', '共和', ' 海南藏族自治州', '门源', ' 海南藏族自治州', '乐都']
['海东市共和', '海东市门源', '海东市乐都']
['海北藏族自治州', ' 海北藏族自治州共和', ' 海北藏族自治州门源', ' 海北藏族自治州乐都']
['阿坝藏族羌族自治州上杜柯作业点', ' 阿坝藏族羌族自治州茸木达作业点', ' 阿坝藏族羌族自治州南木达山顶作业点']
['张掖市山丹县人影办', ' 张掖市各作业点']
['根据目标描述', '“滁州市', '城郊作业点”这一信息中主要包含了城市名称“滁州市”和特定区域描述“城郊作业点”', '由于具体的位置信息并未直接给出', '我们可以理解为这个表述指向了该城市的某个具体的城郊地区作为作业点', '\n\n因此', '按照例子中的输出格式要求', '并基于给定的描述进行转换处理', '“C滁州市', '城郊作业点”可以转换为以下格式', '\n\n**C滁州市城郊作业点**\n\n根据例子1和例子2的输出方式', '我们可以将上述结果转换成', "\n\n**'C滁州市城郊作业点'**\n\n请注意", '由于实际的具体位置信息并未提供', '上述输出已经是按照要求和逻辑做出的最佳解释', '如果有更具体的位置列表或描述', '请提供详细信息以便更精确地进行转换']
['凉山彝族自治州', '冕宁腊窝火场4个作业点', '请返回', '凉山彝族自治州冕宁腊窝火场1号作业点', ' 凉山彝族自治州冕宁腊窝火场2号作业点', ' 凉山彝族自治州冕宁腊窝火场3号作业点', ' 凉山彝族自治州冕宁腊窝火场4号作业点']
['上饶市许村', ' 上饶市中云']
['杭州市百丈', '杭州市鸬鸟', '杭州市径山', '杭州市黄湖', '杭州市良渚', '杭州市塘栖', '杭州市瓶窑', '杭州市仁和以及老余杭']
['聊城市', '菜屯', '贾寨', '博平', '冯屯(王老)', '杜郎口', '乐平(

In [97]:
df_merge_nodup.reset_index(drop=True,inplace=True)
df_merge_nodup['coordinate'] = df_merge_nodup['location']
df_merge_nodup['formatted_address'] = df_merge_nodup['location']
df_merge_nodup['coordinate_level'] = df_merge_nodup['location']

In [98]:
df_merge_nodup

,weibo_text,operation_year,operation_month,operation_day,prov,city,county,area,location,coordinate,coordinate_level,address,formatted_address
0,勐海县抓住有利天气条件 开展人工增雨作业有效缓解旱情记者黎智仙报道： 3月24日...,2020,3,20,云南省,西双版纳傣族自治州,勐海县,星火山、布朗山、勐混、曼稿保护区、勐邦水库,西双版纳傣族自治州星火山,西双版纳傣族自治州星火山,西双版纳傣族自治州星火山,西双版纳傣族自治州星火山,西双版纳傣族自治州星火山
1,勐海县抓住有利天气条件 开展人工增雨作业有效缓解旱情记者黎智仙报道： 3月24日...,2020,3,20,云南省,西双版纳傣族自治州,勐海县,星火山、布朗山、勐混、曼稿保护区、勐邦水库,西双版纳傣族自治州布朗山,西双版纳傣族自治州布朗山,西双版纳傣族自治州布朗山,西双版纳傣族自治州布朗山,西双版纳傣族自治州布朗山
2,勐海县抓住有利天气条件 开展人工增雨作业有效缓解旱情记者黎智仙报道： 3月24日...,2020,3,20,云南省,西双版纳傣族自治州,勐海县,星火山、布朗山、勐混、曼稿保护区、勐邦水库,西双版纳傣族自治州勐混,西双版纳傣族自治州勐混,西双版纳傣族自治州勐混,西双版纳傣族自治州勐混,西双版纳傣族自治州勐混
3,勐海县抓住有利天气条件 开展人工增雨作业有效缓解旱情记者黎智仙报道： 3月24日...,2020,3,20,云南省,西双版纳傣族自治州,勐海县,星火山、布朗山、勐混、曼稿保护区、勐邦水库,西双版纳傣族自治州曼稿保护区,西双版纳傣族自治州曼稿保护区,西双版纳傣族自治州曼稿保护区,西双版纳傣族自治州曼稿保护区,西双版纳傣族自治州曼稿保护区
4,勐海县抓住有利天气条件 开展人工增雨作业有效缓解旱情记者黎智仙报道： 3月24日...,2020,3,20,云南省,西双版纳傣族自治州,勐海县,星火山、布朗山、勐混、曼稿保护区、勐邦水库,西双版纳傣族自治州勐邦水库,西双版纳傣族自治州勐邦水库,西双版纳傣族自治州勐邦水库,西双版纳傣族自治州勐邦水库,西双版纳傣族自治州勐邦水库
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9608,#天气实况#【德州市气象台】强对流天气实况：5月19日20时至20日6时我市北部县（市、区）...,2023,5,19,山东省,德州市,,22个作业点,德州市22,德州市22,德州市22,德州市22,德州市22
9609,风雨正在赶往五莲路上 今夜到明天将实施人工增雨 我县小樱桃正陆续进入盛花期，一阵...,2022,3,16,山东省,日照市,五莲县,16日夜间到17日上午实施人工增雨作业,日照市16日夜间到17日上午实施人工增雨作业,日照市16日夜间到17日上午实施人工增雨作业,日照市16日夜间到17日上午实施人工增雨作业,日照市16日夜间到17日上午实施人工增雨作业,日照市16日夜间到17日上午实施人工增雨作业
9610,风雨正在赶往五莲路上 今夜到明天将实施人工增雨 我县小樱桃正陆续进入盛花期，一阵...,2022,3,16,山东省,日照市,五莲县,16日夜间到17日上午实施人工增雨作业,返回的具体位置信息为,返回的具体位置信息为,返回的具体位置信息为,返回的具体位置信息为,返回的具体位置信息为
9611,风雨正在赶往五莲路上 今夜到明天将实施人工增雨 我县小樱桃正陆续进入盛花期，一阵...,2022,3,16,山东省,日照市,五莲县,16日夜间到17日上午实施人工增雨作业,日照市,日照市,日照市,日照市,日照市


In [111]:
import requests
import pandas as pd

# 高德地图 API 配置
api_key = '9deaf41d592ad149cdc6f36dbe95e835'
geocode_url = 'https://restapi.amap.com/v3/geocode/geo'

# 通过地理编码获取经纬度
for index, row in df_merge_nodup.loc[8320:].iterrows():
    df_merge_nodup.at[index,'location'] = str(df_merge_nodup.at[index,'prov'])+str(df_merge_nodup.at[index,'location'])
    params = {
        'address': str(row['location']),
        'key': api_key,
        'output': 'JSON'
    }
    response = requests.get(geocode_url, params=params)
    data = response.json()

    if data['status'] == '1' and len(data['geocodes']) > 0:
        df_merge_nodup.at[index, 'coordinate']       = data['geocodes'][0]['location']
        df_merge_nodup.at[index, 'coordinate_level'] = data['geocodes'][0]['level']
        df_merge_nodup.at[index, 'formatted_address']= data['geocodes'][0]['formatted_address']
    else:
        df_merge_nodup.at[index, 'coordinate']       = 'Not Found'
        df_merge_nodup.at[index, 'coordinate_level'] = 'Not Found'
        df_merge_nodup.at[index, 'formatted_address']= 'Not Found'

    # 实时显示进度
    print(index,df_merge_nodup.at[index, 'formatted_address'],
          df_merge_nodup.at[index, 'location'],df_merge_nodup.at[index, 'coordinate'],
          df_merge_nodup.at[index, 'coordinate_level'])

print("Processing complete!")


8320 山西省晋中市榆次区 山西省山西省晋中市榆次区 112.707564,37.698637 区县
8321 山西省晋中市祁县 山西省山西省晋中市祁县 112.335226,37.358425 区县
8322 山西省晋中市寿阳县 山西省晋中市寿阳县 113.176373,37.895191 区县
8323 山西省晋中市灵石县 山西省晋中市灵石县 111.778692,36.848085 区县
8324 山西省晋中市左权县 山西省晋中市左权县 113.379412,37.082746 区县
8325 山西省晋中市和顺县 山西省晋中市和顺县 113.570295,37.329696 区县
8326 山西省晋中市介休市 山西省晋中市介休市 111.916451,37.027538 区县
8327 山西省晋中市平遥县 山西省晋中市平遥县 112.175825,37.189614 区县
8328 山西省晋中市昔阳县 山西省晋中市昔阳 113.707151,37.612577 区县
8329 山西省晋中市 山西省晋中市新绛县 112.752633,37.688006 市
8330 山西省晋中市 山西省晋中市襄垣县 112.752633,37.688006 市
8331 山西省晋中市 山西省晋中市孝义市 112.752633,37.688006 市
8332 山西省晋中市 山西省晋中市文水县 112.752633,37.688006 市
8333 山西省晋中市榆社县 山西省晋中市榆社县 112.975378,37.071209 区县
8334 山西省晋中市寿阳县 山西省晋中市寿阳 113.176373,37.895191 区县
8335 山西省晋中市祁县 山西省晋中市祁县 112.335226,37.358425 区县
8336 山西省晋中市灵石县 山西省晋中市灵石县 111.778692,36.848085 区县
8337 山西省晋中市左权县 山西省晋中市左权县 113.379412,37.082746 区县
8338 山西省晋中市和顺县 山西省晋中市和顺县" 113.570295,37.329696 区县
8339 Not Found 湖南省娄底市206个地面作业站点的具体位置信息无法直接提供 Not Found Not Found
8340 Not Found 湖南省因

In [112]:
df_merge_nodup.to_csv('df_merge_nodup_coord.csv')